# Methodology and Analysis Notebook
#### In this notebook I note my methods and ideas as I go

#### Experiment-setup:

I am trying to investigate whether firms can utilize greenwashing as a profitable strategy. In order to do that I want to create a measure of greenwashing by following *(Lagasio, 2024)*. Furthermore, I will utilize a natural experiment (DiD) to search for causality.

<br>

#### Experiment notes:
* Use a DiD with continuous treatment to see whether greenwashing more makes you profit more
* For applying treatment, use the level of greenwashing from 2014/2015
    * The justification is that in 2013/2014 the previous IPCC report was released 
    * Since that created a bit of buzz, it may have allowed firms to start greenwashing
    * I could also just use the classification just before the 2018/2019 shock as that's closer & more data
    * The 2009/2010 shock is explained by the Copenhagen Accord signed in the fall 2009
    <div>
    <img src="../img/ipcc_trends_2008_2021.png" width="600"/>
    </div>

* For the shock, use September 2019, as that's when there was a lot of buzz around another IPCC report and Fridays For Future movement
    <div>
    <img src="../img/climate_trends_2004_2022.png" width="600"/>
    </div>

* use cumulative abnormal returns as the outcome variable 
* use sales as a secondary outcome variable

----
#### Some links:
* [IPPC Wikipedia Page](https://en.wikipedia.org/wiki/Intergovernmental_Panel_on_Climate_Change)
* [6th IPCC Assessment](https://en.wikipedia.org/wiki/IPCC_Sixth_Assessment_Report)
* [IPCC Special Report from 2019](https://en.wikipedia.org/wiki/Special_Report_on_Climate_Change_and_Land)
* [Fridays For Future movement Wiki](https://en.wikipedia.org/wiki/Fridays_for_Future)
* [List of school climate strikes](https://en.wikipedia.org/wiki/List_of_school_climate_strikes)


Total Return Index from Datastream (commented out)
<!-- > Total Return Index from data_greenwashingstream:
> The Return Index shows the growth in value of a security over a specified period, assuming that dividends are re-invested to purchase additional shares of the security at the closing price (P) on the dividend ex-date.
> 
> For all markets except Canada and the US, dividend payment data (DDE) is only available from 1988, for securities listed before this point the return index is calculated using the dividend yield (DY). This method adds an increment of 1/260th part of the dividend yield to the price each weekday. There are assumed to be 260 weekdays in a year, market holidays are ignored.
> 
> Method 1 (using annualised dividend yield)
> 
> RI on the BDATE =100, then:
> 
> cid:image001.gif@01CC34D5.05D81F30
> 
> Where:
> 
> cid:image002.gif@01CC34D5.05D81F30 = return index on day t
> 
> cid:image003.gif@01CC34D5.05D81F30= return index on previous day
> 
> cid:image004.gif@01CC34D5.05D81F30 = price index on day t
> 
> cid:image005.gif@01CC34D5.05D81F30 = price index on previous day
> 
> cid:image006.gif@01CC34D5.05D81F30 = dividend yield % on day t
> 
>  N = number of working days in the year (taken to be 260)
> 
> For securities listed before 1988 the RI calculation switches from the dividend yield method to using the dividend payment data, for example the first recorded dividend for BP is 15/08/88 – on this date the RI calculation switches from the dividend yield method to using the dividend payment data. This method represents a more accurate measure of the security’s growth in which the discrete quantity of dividend paid is added to the price on the ex-date of the payment
> 
> Method 2 (using dividend payment data)
> 
> RI on the BDATE =100, then:
> 
>  cid:image007.gif@01CC34D5.05D81F30
> 
>  except when t = ex-date of the dividend payment Dt then:
> 
> cid:image011.gif@01CC34D6.0623B1B0
> 
> Where:
> 
>  cid:image008.gif@01CC34D5.05D81F30 = price on ex-date
> 
> = price on previous day
> 
>  cid:image010.gif@01CC34D5.05D81F30 = dividend payment associated with ex-date t
> 
> The calculation ignores tax and re-investment charges.
> 
> Canadian and US securities are not affected by the dual calculation method, dividend payment data is available from 1973 – the Datastream market inception date for both markets.
> 
> Securities listed after 1988 automatically use RI calculation method 2, dividend payment data.
> 
> Adjusted closing prices are used throughout to determine price index and hence return index.
> 
> Note: where the dividend payment data history contains a mixture of gross and net dividends the annualised dividend yield (method 1 above) is used in order to achieve a consistent growth measure. The net and gross markers can be identified using the datatype DTAX (tax marker). To display the total return using the dividend payment data (method 2) in these cases, the alternative total return datatype RZ (Return Index - As Paid) may be used. RZ uses the dividend payment data calculation method irrespective of the tax markers.
> 
> For UK companies the RI includes a tax credit on the dividend until it was abolished in April 2004. Prior to that time dividends as announced by the company are grossed up to include to the credit in the RI calculation. The rate used varies over time, the last rate being 10% in the period April 1999 to April 2004. To see a return index measure which does not include tax credits, the datatype RN (Return Index – Net) should be used. The RN calculation is the same as RI except the dividends used are as announced by the company and not grossed up for the tax credits. It follows that since April 2004 the performance of RI and RN for UK securities is the same. RN is available for UK securities only. -->



#### Data Processing

First reading in pre-processed data

In [1]:
# libs
import pandas as pd
import numpy as np
import pymupdf
from datetime import datetime
import os
import statsmodels.api as sm
from itertools import product
import spacy
from funcs import ExtractNameYear, ExtractFileName, ExtractAllText
import pickle 

In [2]:
# data
data_greenwashing = pd.read_excel('../data/LSEG data/matched_final.xlsx', sheet_name='companies')
data_returns = pd.read_excel('../data/LSEG data/matched_final.xlsx', sheet_name='indicators_1')
data_indices = pd.read_excel('../data/LSEG data/matched_final.xlsx', sheet_name='INDICES')

In [ ]:
# processing returns and merging to greenwashing
data_returns = data_returns[(data_returns['Retrieving...'].str.contains("PRICE INDEX")) | (data_returns['Retrieving...'].str.contains("TOT RETURN IND"))]
data_returns.loc[:,"NAME"] = data_returns.loc[:,"Retrieving..."].str.removesuffix(' - PRICE INDEX').str.removesuffix(" - TOT RETURN IND")
data_returns.loc[:, "VARIABLE"] = np.where(data_returns.loc[:,"Retrieving..."].str.contains(" - PRICE INDEX"), 'PRICE INDEX', pd.NA)
data_returns.loc[:, "VARIABLE"] = np.where(data_returns.loc[:,"Retrieving..."].str.contains(" - TOT RETURN IND"), 'TOT RETURN IND', data_returns.loc[:, "VARIABLE"])


data_returns.drop(["Retrieving..."], axis=1, inplace=True)
data_returns = pd.melt(data_returns, id_vars=['NAME', "VARIABLE"]).pivot_table(index=['NAME', 'variable'], columns='VARIABLE', values='value').reset_index().rename(columns={'variable':'DATE'})
data_returns = data_returns.merge(data_greenwashing[['NAME', 'CTRY_OF_DOM_NAME']], how="left", on="NAME")

data_returns["DATE"] = data_returns["DATE"].dt.date



In [ ]:

data_indices.loc[:,"NAME"] = data_indices.loc[:,"Retrieving....1"].str.removesuffix(' - PRICE INDEX').str.removesuffix(" - TOT RETURN IND")
data_indices.loc[:, "VARIABLE"] = np.where(data_indices.loc[:,"Retrieving....1"].str.contains(" - PRICE INDEX"), 'PRICE INDEX', pd.NA)
data_indices.loc[:, "VARIABLE"] = np.where(data_indices.loc[:,"Retrieving....1"].str.contains(" - TOT RETURN IND"), 'TOT RETURN IND', data_indices.loc[:, "VARIABLE"])

data_indices.drop(columns=['Retrieving...', 'Retrieving....1', 'ISIN CODE'], inplace=True)

data_indices = pd.melt(data_indices, id_vars=['NAME', "VARIABLE"]).pivot_table(index=['NAME', 'variable'], columns='VARIABLE', values='value').reset_index().rename(columns={'variable':'DATE'})

data_indices["DATE"] = data_indices["DATE"].dt.date


#### Calculating abnormal returns

Here I calculate abnormal returns and cumulative abnormal returns. In order to do that, I benchmark the realized returns of each of the companies against their expected return. The market model is estimated in order to assess the expected return. The market for each security is chosen with respect to the primary market of operations of each company (S&P for the USA/Canada, MSCI EUROPE for Europe, ... **Work-in-progress, need to get indices for more geographical regions**). Finally, I chose to use the price return on equity only (instead of total return including re-invested dividends - they don't really matter over such a short period anyway)

In [ ]:
# check to see the market from which most of the companies come from ("Country of Domicile")
print(data_greenwashing['CTRY_OF_DOM_NAME'].value_counts(normalize=True))

# creating monthly returns for equities
data_returns.dropna(subset='PRICE INDEX', inplace=True)
data_returns['STOCK_PCT_RETURN'] = data_returns.groupby(by='NAME')['PRICE INDEX'].apply(pd.Series.pct_change).reset_index()['PRICE INDEX']
data_returns = data_returns[['NAME', 'CTRY_OF_DOM_NAME', 'DATE','STOCK_PCT_RETURN']]

# making sure that all firms have data starting from 2014
full_index = pd.DataFrame(product(data_returns["NAME"].unique(), data_returns["DATE"].unique()))
full_index.columns = ["NAME", "DATE"]

data_returns = full_index.merge(data_returns, how="left", on=['NAME', 'DATE'])

# creating monthly returns for indices
data_indices['INDEX_PCT_RETURN'] = data_indices.groupby(by='NAME')['PRICE INDEX'].apply(pd.Series.pct_change).reset_index()['PRICE INDEX']
data_indices = data_indices[['NAME', 'DATE','INDEX_PCT_RETURN']]
data_indices.dropna(subset='INDEX_PCT_RETURN', inplace=True)


Checking for duplicated data in the companies. It seems that some have been duplicated but mostly due to having a subsidiary in the same name/having multiple pages etc. 
Some of them are also the wrong company. There's little enough problems though, that it seems like it could be dropped (16 companies like that) 

In [ ]:
print(data_returns[['NAME', 'DATE']].value_counts())

data_returns.drop_duplicates(subset=['NAME', 'DATE'], inplace=True)


In [ ]:
# since 2014-01-01 was the initial time period, I'm dropping it
data_returns = data_returns[data_returns["DATE"] != datetime.strptime('2014-01-01', '%Y-%m-%d').date()]

# set the estimation window
date_start = datetime.strptime('2010-01-01', '%Y-%m-%d').date()
date_end = datetime.strptime('2018-09-01', '%Y-%m-%d').date()

# company related parameters
company_array = data_returns[['NAME', 'CTRY_OF_DOM_NAME']].drop_duplicates().values
market_dict = {
                # temporarily assigned to S&P 500
                "UNITED STATES":"S&P 500 COMPOSITE",
                "CANADA":"S&P 500 COMPOSITE",
                "BERMUDA":"S&P 500 COMPOSITE",
                "CAYMAN ISLANDS":"S&P 500 COMPOSITE",
                
                # temporarily assigned to MSCI europe
                "UNITED KINGDOM":"MSCI EUROPE U$",
                "IRELAND":"MSCI EUROPE U$",
                "SWITZERLAND":"MSCI EUROPE U$",
                "NETHERLANDS":"MSCI EUROPE U$",
                "GREECE":"MSCI EUROPE U$",
                "GERMANY":"MSCI EUROPE U$",
                "BELGIUM":"MSCI EUROPE U$",
                "DENMARK":"MSCI EUROPE U$",
                "MONACO":"MSCI EUROPE U$",
                "LUXEMBOURG":"MSCI EUROPE U$",
                "FRANCE":"MSCI EUROPE U$",
                "SWEDEN":"MSCI EUROPE U$",
                "ISLE OF MAN":"MSCI EUROPE U$",
                "SPAIN":"MSCI EUROPE U$",
                "FINLAND":"MSCI EUROPE U$",
                "ROMANIA":"MSCI EUROPE U$",
                "ITALY":"MSCI EUROPE U$",
                "AUSTRIA":"MSCI EUROPE U$",
                "JERSEY":"MSCI EUROPE U$",
                "GUERNSEY":"MSCI EUROPE U$",
                
                # temporarily assigned to MSCI world
                "MEXICO":"MSCI WORLD U$",
                "PAPUA NEW GUINEA":"MSCI WORLD U$",
                "PUERTO RICO":"MSCI WORLD U$",
                "PANAMA":"MSCI WORLD U$",
                "COLOMBIA":"MSCI WORLD U$",
                "BRAZIL":"MSCI WORLD U$",
                "CHILE":"MSCI WORLD U$",
                "PERU":"MSCI WORLD U$",
                "URUGUAY":"MSCI WORLD U$",
                "ARGENTINA":"MSCI WORLD U$",
                "COSTA RICA":"MSCI WORLD U$",
                "TURKEY":"MSCI WORLD U$",
                "ISRAEL":"MSCI WORLD U$",
                "KAZAKHSTAN":"MSCI WORLD U$",
                "UNITED ARAB EMIRATES":"MSCI WORLD U$",
                "SOUTH AFRICA":"MSCI WORLD U$",
                "INDIA":"MSCI WORLD U$",
                "TAIWAN":"MSCI WORLD U$",
                "JAPAN":"MSCI WORLD U$",
                "SOUTH KOREA":"MSCI WORLD U$",
                "SINGAPORE":"MSCI WORLD U$",
                "MONGOLIA":"MSCI WORLD U$",
                "CHINA":"MSCI WORLD U$",
                "HONG KONG":"MSCI WORLD U$",
                "INDONESIA":"MSCI WORLD U$",
                "PHILIPPINES":"MSCI WORLD U$",
                "BARBADOS":"MSCI WORLD U$",
                "AUSTRALIA":"MSCI WORLD U$",
                "NEW ZEALAND":"MSCI WORLD U$",
                
                # missing values
                np.nan:'NA'
               
    
}



data_abnormal_returns = data_returns[(data_returns["DATE"] >= date_end)]
data_abnormal_returns['NORMAL_RETURN'] = pd.NA

for company_list in company_array:
    company, cntry = company_list
    
    
    print(company)
    print(cntry)
    
    if (market_dict[cntry] != "S&P 500 COMPOSITE"):
        print(f"Skip {company} because no S&P")
        continue
    
    
    est_data_comp = data_returns[(data_returns['NAME']==company) & (data_returns["DATE"] > date_start) & (data_returns["DATE"] < date_end)]
    est_data_ind = data_indices[(data_indices['NAME'] == market_dict[cntry]) & (data_indices['DATE'] > date_start) & (data_indices['DATE']< date_end)].reset_index(drop=True).set_index(est_data_comp.index)
    est_data_ind = sm.add_constant(est_data_ind)
    
    if  est_data_comp["STOCK_PCT_RETURN"].isna().sum()>0:
        print(f"Company: {company} skipped because NAs")
        continue 
    
    reg = sm.OLS(est_data_comp['STOCK_PCT_RETURN'], est_data_ind[['const', "INDEX_PCT_RETURN"]]).fit(cov_type="HC3")
    
    pred_data_ind = data_indices[(data_indices['NAME'] == market_dict[cntry]) & (data_indices['DATE'] >= date_end)].reset_index(drop=True)
    pred_data_ind = sm.add_constant(pred_data_ind)
    
    
    predicted = reg.predict(pred_data_ind[['const', 'INDEX_PCT_RETURN']])
    predicted.name = 'NORMAL_RETURN'
    predicted = predicted.set_axis(data_abnormal_returns.loc[data_abnormal_returns['NAME']== company, 'NORMAL_RETURN'].index)
    
    # print(reg.summary())
    
    
    data_abnormal_returns.loc[data_abnormal_returns['NAME']== company, 'NORMAL_RETURN'] = predicted
    
    


# NOTE NEED TO APPROACH THIS AGAIN - MAYBE USE STOCK PRICES AT THE DAILY LEVEL?      
# MANY COMPANIES DON'T HAVE DATA THAT STARTS ENOUGH MONTHS BEFORE THE SHOCK SO MAYBE THE DAILY DATA WOULD BE BETTER 
# THEN I WOULD NEED TO CHOOSE A SPECIFIC DATE BETWEEN THE IPCC REPORT AND THE START OF THE CLIMATE WEEK
#     
# 


#### Greenwashing Indicator

In [3]:
# /////////////////////////////////////
#       expanding
# /////////////////////////////////////

data_greenwashing['REPORT_LISTS'] = data_greenwashing['REPORT_LISTS'].apply(eval)

# Expand each list entry into its own row
data_greenwashing = data_greenwashing.explode('REPORT_LISTS', ignore_index=True)

data_greenwashing['YEAR'] = data_greenwashing['REPORT_LISTS'].str.extract(r'(\d+)')
data_greenwashing['YEAR'] = data_greenwashing['YEAR'].astype(float)

print(data_greenwashing.value_counts('YEAR'))

obs_2014 = data_greenwashing[data_greenwashing['YEAR']==2014]

# # ////////////////////////////////////////////////////////
# # NOTE HOW MANY FIRMS WITH REPORTS IN THE YEARS I NEED
g = data_greenwashing[(data_greenwashing["YEAR"]>2016) & (data_greenwashing["YEAR"]<2022)].groupby("NAME_SCRAPED").count()
print(g.value_counts("ICB_INDUSTRY_NAME"))


YEAR
2022.0     2020
2023.0     1859
2021.0     1733
2020.0     1355
2019.0     1081
2018.0      852
2017.0      686
2016.0      558
2015.0      475
2014.0      415
2013.0      357
2012.0      300
2011.0      252
2010.0      216
2009.0      177
2008.0      147
2007.0      121
2006.0       86
2005.0       71
2024.0       60
2004.0       54
2003.0       48
2002.0       35
2001.0       23
2000.0       17
1999.0        8
1998.0        7
20111.0       1
201.0         1
1989.0        1
1997.0        1
1996.0        1
1995.0        1
1994.0        1
1993.0        1
1992.0        1
20167.0       1
Name: count, dtype: int64
ICB_INDUSTRY_NAME
1    444
5    441
2    303
3    241
0    239
4    204
6      1
Name: count, dtype: int64


Text extraction

In [4]:
# get datasets that have a 2017 sustainability report

greenwashing_2017 = data_greenwashing[data_greenwashing["YEAR"]==2017]

Getting all 2017 sustainability reports

In [5]:
nlp = spacy.load('en_core_web_sm')

test = greenwashing_2017["NAME_SCRAPED"]
input_name = "C:/Users/Jakub/OneDrive - Tilburg University/thesis data/responsibility reports"

filenames = ExtractFileName(series_names=test, input_dir=input_name)

test_updated = test.str.lower().str.replace(" ", "_").str.replace("?", "").str.replace("|", "")
test_updated = test_updated.loc[~(test_updated.isin(filenames.keys()))]

filenames_updated = ExtractNameYear(series_names=test_updated, input_dir=input_name, nlp_model=nlp)

filenames.update(filenames_updated)

Dumping all of the text from PDF files into their respective TXT files for later processing

In [11]:
ExtractAllText(filenames, "2017", "../data/text_processing")

NLP model and greenwashing indicator